# Multiple Choice Question

### 1. Import libraries

In [1]:
import evaluate
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict

### 2. Load data

In [2]:
c3 = DatasetDict.load_from_disk("../c3/")

`DatasetDict.load_from_disk("")`
Purpose: Load a dataset (or dataset dictionary) that was previously saved locally with `.save_to_disk()`.

This method directly restores the dataset exactly as it was saved, including:

- Column formats
- Features schema
- Train/validation/test splits (if it was a DatasetDict)

No processing is done — it just reopens the existing preprocessed dataset.

In [3]:
c3

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 1625
    })
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [4]:
c3["train"][3:5]

{'id': [3, 4],
 'context': [['男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?', '女：是啊，我希望自己能瘦一点儿。'],
  ['女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。',
   '男：可不是嘛!自从我去日本留学，就再也没见过他，算一算都五年了。',
   '女：从2000年我们在学校第一次见面到现在已经快十年了。我还真想看看刘明变成什么样了!',
   '男：你还别说，刘明肯定跟英国绅士一样，也许还能带回来一个英国女朋友呢。']],
 'question': ['女的为什么不吃巧克力了?', '现在大概是哪一年?'],
 'choice': [['刷牙了', '要减肥', '口渴了', '吃饱了'],
  ['2005年', '2010年', '2008年', '2009年']],
 'answer': ['要减肥', '2010年']}

In [5]:
c3["test"][3:5]

{'id': [3, 4],
 'context': [['这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之后都开始担心，以为要离开的是自己。后来经理宣布了一个消息：大家都是一家人，虽然公司有困难，但不能丢掉任何人，只是没有奖金了。这个消息使所有的人都放下了心：奖金不重要，有工作就好。春节快到了，工人们都做了过个穷年的打算。这时经理通知开会，工人们又担心：“会有什么变化吗？”谁知参加会议的人回来兴奋地喊道：“有！有！还是有奖金的！一个月的！”工人们听了，发出一片热烈的欢呼声。'],
  ['这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之后都开始担心，以为要离开的是自己。后来经理宣布了一个消息：大家都是一家人，虽然公司有困难，但不能丢掉任何人，只是没有奖金了。这个消息使所有的人都放下了心：奖金不重要，有工作就好。春节快到了，工人们都做了过个穷年的打算。这时经理通知开会，工人们又担心：“会有什么变化吗？”谁知参加会议的人回来兴奋地喊道：“有！有！还是有奖金的！一个月的！”工人们听了，发出一片热烈的欢呼声。']],
 'question': ['今年公司怎么样？', '根据经理小时候买糖的情况，可以知道？'],
 'choice': [['发展得不错', '挣的钱不多', '要给工人发糖', '要发两个月的奖金'],
  ['服务员给的重量不够', '服务员一块儿一块儿地卖', '服务员卖糖的方式都一样', '不同的卖糖方法影响人的心情']],
 'answer': ['', '']}

Test data doesn't have values in 'answer'. In the preprocessing of data, we remove "test" first.

In [6]:
c3.pop("test")

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer'],
    num_rows: 1625
})

In [7]:
c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

### 3. Preprocess the data

In [8]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [9]:
tokenizer

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [10]:
c3["train"][10]

{'id': 10,
 'context': ['女：每次考试之前我都睡不着，所以考试的时候精神不好。', '男：那是因为你太紧张了，你在睡觉之前听点儿轻松的音乐吧。'],
 'question': '关于女的，可以知道什么?',
 'choice': ['考得好', '睡不着', '很轻松', '觉得累'],
 'answer': '睡不着'}

In [11]:
def process_function(examples):
    # examples["context", "question", "choice", "answer"]
    context = [] #sentence1
    question_choice = [] #sentence2
    labels = []
    for idx in range(len(examples["context"])):
        ctx = "\n".join(examples["context"][idx])
        question = examples["question"][idx]
        choices = examples["choice"][idx]
        for choice in choices:
            context.append(ctx)
            question_choice.append(question + " " + choice)
        if len(choices) <4:
            for _ in range(4-len(choices)):
                context.append(ctx)
                question_choice.append(question + " " + "不知道")

        labels.append(choices.index(examples["answer"][idx]))

    # 1000*4 = 4000 -> 4000*256
    tokenized_examples = tokenizer(context, question_choice, truncation = "only_first", max_length = 256, padding = "max_length")
    
    # 1000*4*256
    tokenized_examples = {k: [v[i: i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
    tokenized_examples["labels"] = labels
    
    return tokenized_examples
    

In [12]:
res = c3["train"].select(range(10)).map(process_function, batched = True)

In [13]:
import numpy as np
np.array(res["input_ids"]).shape

(10, 4, 256)

In [14]:
print(res[3:5])

{'id': [3, 4], 'context': [['男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?', '女：是啊，我希望自己能瘦一点儿。'], ['女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。', '男：可不是嘛!自从我去日本留学，就再也没见过他，算一算都五年了。', '女：从2000年我们在学校第一次见面到现在已经快十年了。我还真想看看刘明变成什么样了!', '男：你还别说，刘明肯定跟英国绅士一样，也许还能带回来一个英国女朋友呢。']], 'question': ['女的为什么不吃巧克力了?', '现在大概是哪一年?'], 'choice': [['刷牙了', '要减肥', '口渴了', '吃饱了'], ['2005年', '2010年', '2008年', '2009年']], 'answer': ['要减肥', '2010年'], 'input_ids': [[[101, 4511, 8038, 2769, 6381, 2533, 872, 809, 1184, 2523, 4263, 1391, 2341, 1046, 1213, 8024, 3297, 6818, 2582, 720, 679, 1391, 749, 8024, 3221, 1762, 1121, 5503, 1408, 136, 1957, 8038, 3221, 1557, 8024, 2769, 2361, 3307, 5632, 2346, 5543, 4607, 671, 4157, 1036, 511, 102, 1957, 4638, 711, 784, 720, 679, 1391, 2341, 1046, 1213, 749, 136, 1170, 4280, 749, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
tokenized_c3 = c3.map(process_function, batched=True)
tokenized_c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3816
    })
})

### 4. Create model

In [16]:
model = AutoModelForMultipleChoice.from_pretrained("hfl/chinese-macbert-base")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
for param in model.parameters():
    param.data = param.data.contiguous()

### 5. Create evaluation function

In [18]:
accuracy = evaluate.load("accuracy")

In [19]:
def compute_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis = -1)
    return accuracy.compute(predictions = predictions, references = labels)

### 6. Set up training configuration

In [20]:
args = TrainingArguments(
    output_dir = "./mcq",
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 2,
    logging_steps = 10,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    # fp16 = True,
    # bf16=use_mps,
    disable_tqdm = True
)

In [21]:
args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=True,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
eval_use_gather_object=False,
fp16=False,
fp16

In [22]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset = tokenized_c3["train"].select(range(3000)),
    eval_dataset = tokenized_c3["validation"].select(range(1000)),
    compute_metrics = compute_metric
)

/var/folders/sj/km6fyx_57pn857bnxsqvkj_80000gn/T/ipykernel_92952/2621065752.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
trainer.train()

/opt/anaconda3/envs/Advanced_AI/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/Advanced_AI/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.4087, 'grad_norm': 3.423954486846924, 'learning_rate': 4.94e-05, 'epoch': 0.02666666666666667}
{'loss': 1.3442, 'grad_norm': 9.257855415344238, 'learning_rate': 4.8733333333333337e-05, 'epoch': 0.05333333333333334}
{'loss': 1.2155, 'grad_norm': 12.889373779296875, 'learning_rate': 4.806666666666667e-05, 'epoch': 0.08}
{'loss': 1.2541, 'grad_norm': 21.959035873413086, 'learning_rate': 4.74e-05, 'epoch': 0.10666666666666667}
{'loss': 1.3318, 'grad_norm': 10.507397651672363, 'learning_rate': 4.6733333333333335e-05, 'epoch': 0.13333333333333333}
{'loss': 1.2537, 'grad_norm': 21.981111526489258, 'learning_rate': 4.606666666666667e-05, 'epoch': 0.16}
{'loss': 1.1767, 'grad_norm': 14.444607734680176, 'learning_rate': 4.5400000000000006e-05, 'epoch': 0.18666666666666668}
{'loss': 1.1926, 'grad_norm': 18.59183120727539, 'learning_rate': 4.473333333333334e-05, 'epoch': 0.21333333333333335}
{'loss': 1.2286, 'grad_norm': 12.094952583312988, 'learning_rate': 4.406666666666667e-05, 'epoch

/opt/anaconda3/envs/Advanced_AI/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/Advanced_AI/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.9445, 'grad_norm': 19.225492477416992, 'learning_rate': 2.4733333333333333e-05, 'epoch': 1.0133333333333334}
{'loss': 0.8057, 'grad_norm': 33.72003173828125, 'learning_rate': 2.4066666666666668e-05, 'epoch': 1.04}
{'loss': 0.7116, 'grad_norm': 33.935279846191406, 'learning_rate': 2.3400000000000003e-05, 'epoch': 1.0666666666666667}
{'loss': 1.0463, 'grad_norm': 21.617469787597656, 'learning_rate': 2.2733333333333335e-05, 'epoch': 1.0933333333333333}
{'loss': 0.748, 'grad_norm': 29.042076110839844, 'learning_rate': 2.206666666666667e-05, 'epoch': 1.12}
{'loss': 0.7569, 'grad_norm': 16.382770538330078, 'learning_rate': 2.1400000000000002e-05, 'epoch': 1.1466666666666667}
{'loss': 0.7629, 'grad_norm': 16.872390747070312, 'learning_rate': 2.0733333333333334e-05, 'epoch': 1.1733333333333333}
{'loss': 0.8266, 'grad_norm': 10.434231758117676, 'learning_rate': 2.0066666666666665e-05, 'epoch': 1.2}
{'loss': 0.8099, 'grad_norm': 14.975920677185059, 'learning_rate': 1.94e-05, 'epoch': 

TrainOutput(global_step=750, training_loss=0.9534867680867513, metrics={'train_runtime': 3565.2596, 'train_samples_per_second': 1.683, 'train_steps_per_second': 0.21, 'train_loss': 0.9534867680867513, 'epoch': 2.0})

### 9. Model prediction

As we still need to process the test data, so we just create a class to incorporate all the procedures.

In [24]:
import torch

In [25]:
class MultipleChoicePipeline:

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, context, question, choices):
        ctx = [] #sentence1
        question_choice = [] #sentence2
        for choice in choices:
            ctx.append(context)
            question_choice.append(question + " " + choice)

        print(f"ctx is {ctx}, \nquestion_choice is {question_choice}")
        return tokenizer(ctx, question_choice, truncation = "only_first", max_length = 256, return_tensors = "pt", padding=True)

    def predict(self, inputs):
        print(f"inputs is {inputs}")
        """
         Hugging Face multiple-choice models expect the input tensors to have three dimensions:
         batch_size, num_choices, seq_length
         If you skip unsqueeze(0), the model will think you're passing 4 separate examples, each with one choice,
         instead of 1 example with 4 choices.
         That breaks the logic of multiple-choice classification, where the model compares all choices within a single example.
        """
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
        print(f"After quick process, inputs is {inputs}")
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        prediction = torch.argmax(logits, dim = -1).cpu().item()
        return choices[prediction]

    def __call__(self, context, question, choices):
        inputs = self.preprocess(context, question, choices)
        logits = self.predict(inputs)
        result = self.postprocess(logits, choices)
        return result

In [26]:
pipe = MultipleChoicePipeline(model, tokenizer)

In [27]:
context_test = "成功依靠机遇，英雄出自挑战。机，就是命运给予我们的机会；遇，就是我们去闯、去感受、去把握。命运是公平的，每棵草上都有一颗露珠。在人生路上，我们会有很多次机遇，或当官、或发财、或发明创造……抓住一个，你就会成功。问题是，我们绝大部分的人总是与机遇擦肩而过，或者在机遇面前只是想想而没有采取行动。到最后，人们总是抱怨命运不公平，认为自己没有出生在一个好的时代。有机遇就有挑战。我们一出生便面临各种挑战：学习走路时，我们无数次地摔倒，却仍然扶养墙站起来继续走；学话时，即使咬到舌头，一个字读了几千遍，也要把音念准；上学后，我们一心一意地读书，十多年如一日，放弃了自己的兴趣和爱好……挑战就是我们遇到的种种困难，我们无法回避，也不应该回避。每战胜一个困难，我们就前进一步，而新的困难又会拦在我们前进的路上，我们别无选择，只有迎难而上。"
question_test = "我们一生中会有多少次机遇？"
choices_test = ["一次","许多次","没有","四次"]

In [28]:
pipe(context_test, question_test, choices_test)

ctx is ['成功依靠机遇，英雄出自挑战。机，就是命运给予我们的机会；遇，就是我们去闯、去感受、去把握。命运是公平的，每棵草上都有一颗露珠。在人生路上，我们会有很多次机遇，或当官、或发财、或发明创造……抓住一个，你就会成功。问题是，我们绝大部分的人总是与机遇擦肩而过，或者在机遇面前只是想想而没有采取行动。到最后，人们总是抱怨命运不公平，认为自己没有出生在一个好的时代。有机遇就有挑战。我们一出生便面临各种挑战：学习走路时，我们无数次地摔倒，却仍然扶养墙站起来继续走；学话时，即使咬到舌头，一个字读了几千遍，也要把音念准；上学后，我们一心一意地读书，十多年如一日，放弃了自己的兴趣和爱好……挑战就是我们遇到的种种困难，我们无法回避，也不应该回避。每战胜一个困难，我们就前进一步，而新的困难又会拦在我们前进的路上，我们别无选择，只有迎难而上。', '成功依靠机遇，英雄出自挑战。机，就是命运给予我们的机会；遇，就是我们去闯、去感受、去把握。命运是公平的，每棵草上都有一颗露珠。在人生路上，我们会有很多次机遇，或当官、或发财、或发明创造……抓住一个，你就会成功。问题是，我们绝大部分的人总是与机遇擦肩而过，或者在机遇面前只是想想而没有采取行动。到最后，人们总是抱怨命运不公平，认为自己没有出生在一个好的时代。有机遇就有挑战。我们一出生便面临各种挑战：学习走路时，我们无数次地摔倒，却仍然扶养墙站起来继续走；学话时，即使咬到舌头，一个字读了几千遍，也要把音念准；上学后，我们一心一意地读书，十多年如一日，放弃了自己的兴趣和爱好……挑战就是我们遇到的种种困难，我们无法回避，也不应该回避。每战胜一个困难，我们就前进一步，而新的困难又会拦在我们前进的路上，我们别无选择，只有迎难而上。', '成功依靠机遇，英雄出自挑战。机，就是命运给予我们的机会；遇，就是我们去闯、去感受、去把握。命运是公平的，每棵草上都有一颗露珠。在人生路上，我们会有很多次机遇，或当官、或发财、或发明创造……抓住一个，你就会成功。问题是，我们绝大部分的人总是与机遇擦肩而过，或者在机遇面前只是想想而没有采取行动。到最后，人们总是抱怨命运不公平，认为自己没有出生在一个好的时代。有机遇就有挑战。我们一出生便面临各种挑战：学习走路时，我们无数次地摔倒，却仍然扶养墙站起来继续走；学话时，即使咬到舌头，一个字读了几千遍，也要把音念准；上学后

/opt/anaconda3/envs/Advanced_AI/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'没有'

In [29]:
context_test = "这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之后都开始担心，以为要离开的是自己。后来经理宣布了一个消息：大家都是一家人，虽然公司有困难，但不能丢掉任何人，只是没有奖金了。这个消息使所有的人都放下了心：奖金不重要，有工作就好。春节快到了，工人们都做了过个穷年的打算。这时经理通知开会，工人们又担心：“会有什么变化吗？”谁知参加会议的人回来兴奋地喊道：“有！有！还是有奖金的！一个月的！”工人们听了，发出一片热烈的欢呼声。"
question_test = "关于经理的办法，正确的是？"
choices_test = [
"让工人都很难过",
"增加了公司的收入",
"使工人得到了安慰",
"使一部分人丢了工作"
]

In [30]:
pipe(context_test, question_test, choices_test)

ctx is ['这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之后都开始担心，以为要离开的是自己。后来经理宣布了一个消息：大家都是一家人，虽然公司有困难，但不能丢掉任何人，只是没有奖金了。这个消息使所有的人都放下了心：奖金不重要，有工作就好。春节快到了，工人们都做了过个穷年的打算。这时经理通知开会，工人们又担心：“会有什么变化吗？”谁知参加会议的人回来兴奋地喊道：“有！有！还是有奖金的！一个月的！”工人们听了，发出一片热烈的欢呼声。', '这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之后都开始担心，以为要离开的是自己。后来经理宣布了一个消息：大家都是一家人，虽然公司有困难，但不能丢掉任何人，只是没有奖金了。这个消息使所有的人都放下了心：奖金不重要，有工作就好。春节快到了，工人们都做了过个穷年的打算。这时经理通知开会，工人们又担心：“会有什么变化吗？”谁知参加会议的人回来兴奋地喊道：“有！有！还是有奖金的！一个月的！”工人们听了，发出一片热烈的欢呼声。', '这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之后都开始担心，以为要离开的是自己。后来经理宣布了一个消息：大家都

'使工人得到了安慰'

### Below is a test from a model with 600 samples and 1 epoch in training.

In [65]:
pipe(context_test, question_test, choices_test)

ctx is ['成功依靠机遇', '成功依靠机遇', '成功依靠机遇', '成功依靠机遇'], 
question_choice is ['我们 一次', '我们 许多次', '我们 没有', '我们 四次']
inputs is {'input_ids': tensor([[ 101, 2768, 1216,  898, 7479, 3322, 6878,  102, 2769,  812,  671, 3613,
          102,    0],
        [ 101, 2768, 1216,  898, 7479, 3322, 6878,  102, 2769,  812, 6387, 1914,
         3613,  102],
        [ 101, 2768, 1216,  898, 7479, 3322, 6878,  102, 2769,  812, 3766, 3300,
          102,    0],
        [ 101, 2768, 1216,  898, 7479, 3322, 6878,  102, 2769,  812, 1724, 3613,
          102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

'一次'

In [66]:
tokenizer.decode([ 101, 2768, 1216,  898, 7479, 3322, 6878,  102, 2769,  812,  671, 3613,
          102,    0])

'[CLS] 成 功 依 靠 机 遇 [SEP] 我 们 一 次 [SEP] [PAD]'